In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
import json
from pandas.api.types import is_numeric_dtype
import re

# 1. Loading the file with possible delimiters and decimals

In [31]:
for delim in (',', '|', ';'):
    for decim in (',', '.'):
        df = pd.read_csv("proj2_data.csv", sep=delim, decimal=decim)
        # print(df)
        if len(df.columns) > 1 and len(df.select_dtypes(include='float').columns) > 0:
            break
    else:
        continue
    break
df.to_pickle("proj2_ex01.pkl")
df

,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,python,wej,3.1,2.0,4.4,3.166667,dostateczny,"3,5 pts",dostateczny
1,Nash Wyatt,racing,java,sfe,4.2,2.0,2.0,2.733333,bardzo dobry,5 p,mierny
2,Jadiel Ramirez,media,cplusplus,vaw,4.0,4.9,3.0,3.966667,dobry,3.5,mierny
3,Makaila Atkins,racing,swift,ugt,4.1,5.0,4.8,4.633333,dobry,2,dostateczny
4,Melanie Fuller,racing,python,owb,2.7,2.0,2.0,2.233333,bardzo dobry,pts 2,mierny
5,Layla Woodard,drones,python,wuf,2.8,2.0,2.0,2.266667,bardzo dobry,"3,5p",dostateczny
6,Edwin Hale,media,swift,ceq,4.9,3.0,3.4,3.766667,dostateczny,-1 (dnf),niedostateczny
7,Travis Rocha,robotics,cplusplus,eir,5.0,4.9,5.0,4.966667,dostateczny,0,niedostateczny
8,Ricky Howell,media,java,vos,2.1,2.0,2.0,2.033333,dobry,waiting,niedostateczny
9,Addyson Brennan,drones,java,cov,4.4,3.7,4.0,4.033333,bardzo dobry,-,niedostateczny


# 2. Value scale

In [32]:
scale = {}
file1 = open("proj2_scale.txt", "r")
i = 1
for row in file1:
    scale[row.strip()] = i
    i +=1
print(scale)

{'niedostateczny': 1, 'mierny': 2, 'dostateczny': 3, 'dobry': 4, 'bardzo dobry': 5}


In [33]:
df1 = df.copy()
for column in df1:
    if len(df1[~df1[column].isin(scale.keys())]) == 0:
        df1[column].replace(scale, inplace=True)
df1.to_pickle('proj2_ex02.pkl')
df1

,full_name,field,language,code,task_1,task_2,task_3,tasks_avg,task_grade,jury_score,final_grade
0,Rowan Harrington,drones,python,wej,3.1,2.0,4.4,3.166667,3,"3,5 pts",3
1,Nash Wyatt,racing,java,sfe,4.2,2.0,2.0,2.733333,5,5 p,2
2,Jadiel Ramirez,media,cplusplus,vaw,4.0,4.9,3.0,3.966667,4,3.5,2
3,Makaila Atkins,racing,swift,ugt,4.1,5.0,4.8,4.633333,4,2,3
4,Melanie Fuller,racing,python,owb,2.7,2.0,2.0,2.233333,5,pts 2,2
5,Layla Woodard,drones,python,wuf,2.8,2.0,2.0,2.266667,5,"3,5p",3
6,Edwin Hale,media,swift,ceq,4.9,3.0,3.4,3.766667,3,-1 (dnf),1
7,Travis Rocha,robotics,cplusplus,eir,5.0,4.9,5.0,4.966667,3,0,1
8,Ricky Howell,media,java,vos,2.1,2.0,2.0,2.033333,4,waiting,1
9,Addyson Brennan,drones,java,cov,4.4,3.7,4.0,4.033333,5,-,1


# 3. Categories

In [39]:
df2 = df.copy()
for column in df2:
    if len(df2[~df2[column].isin(scale.keys())]) == 0:
        df2[column] = pd.Categorical(df2[column], categories=scale.keys())
df2.to_pickle('proj2_ex03.pkl')
df2.dtypes

full_name        object
field            object
language         object
code             object
task_1          float64
task_2          float64
task_3          float64
tasks_avg       float64
task_grade     category
jury_score       object
final_grade    category
dtype: object

# 4. Number extraction

In [40]:
df3 = pd.DataFrame()
for col in df:
    # print(df[col].apply(lambda val: val.isnumeric()).all())
    if df.dtypes[col]!='float' and df[col].apply(lambda val: any(ch.isdigit() for ch in val)).any():
        df3[col] = df[col]
        # for var in df3[col]:
        for i in range(df.shape[0]):
            var = df3[col][i]
            print(var)
            match = re.search("-?\d+(\.\d+|\,\d+)?", var)
            if match:
                if re.search("\,|\.", match.group(0)):
                    var = float(match.group(0).replace(',','.'))
                    # print(var)
                else:
                    var = int(match.group(0))
                    # print(var)
            else:
                var = match
            df3.loc[i, col] = var
print(df3)
df3.to_pickle("proj2_ex04.pkl")

3,5 pts
  5 p
3.5
2
pts 2
3,5p
-1 (dnf)
0
waiting
-
2
4
   jury_score
0         3.5
1           5
2         3.5
3           2
4           2
5         3.5
6          -1
7           0
8        None
9        None
10          2
11          4


# 5. One-hot encoding

In [41]:
i = 1
for col in df:
    list = []
    if df[col].apply(lambda val: re.search("[a-z]*", val).group(0) and val not in scale.keys()).all():
        for val in df[col]:
            if val not in list:
                list.append(val)
    if len(list) > 10:
        break
    if list:
        print(list)
        one_hot_encoded_data = pd.get_dummies(df[col]) 
        print(one_hot_encoded_data)
        one_hot_encoded_data.to_pickle(f'proj2_ex05_{i}.pkl')
        i += 1


['drones', 'racing', 'media', 'robotics']
    drones  media  racing  robotics
0     True  False   False     False
1    False  False    True     False
2    False   True   False     False
3    False  False    True     False
4    False  False    True     False
5     True  False   False     False
6    False   True   False     False
7    False  False   False      True
8    False   True   False     False
9     True  False   False     False
10    True  False   False     False
11   False  False    True     False
['python', 'java', 'cplusplus', 'swift']
    cplusplus   java  python  swift
0       False  False    True  False
1       False   True   False  False
2        True  False   False  False
3       False  False   False   True
4       False  False    True  False
5       False  False    True  False
6       False  False   False   True
7        True  False   False  False
8       False   True   False  False
9       False   True   False  False
10      False  False   False   True
11      False  Fa